In [4]:
import pandas as pd
data = pd.read_csv('Train.csv', sep=',')
print("Shape of the data", data.shape)
data.head()

Shape of the data (8523, 12)


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [112]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [5]:
## Check for duplicate rows and if exists drop the row
duplicate_rows = data.duplicated()
duplicate_rows.sum()

np.int64(0)

In [6]:
## Check for null values
data.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

### Handling null values in columns Item_weight and Outlet_size

In [7]:
unique_items = data['Item_Identifier'].unique()
non_null_item_weights = data.dropna(axis=0,how='any',subset=['Item_Weight'])
unique_items_with_weight = non_null_item_weights['Item_Identifier'].unique()

print("Shape of the non null itew weights", non_null_item_weights.shape)
print("Unique items in data", unique_items.shape)
print("Unique items without weight", unique_items_with_weight.shape)

Shape of the non null itew weights (7060, 12)
Unique items in data (1559,)
Unique items without weight (1555,)


In [8]:
unique_items_weight = { item : data.loc[data['Item_Identifier'] == item, 'Item_Weight'].iloc[0] for item in unique_items }
unique_items_weight
data['Item_Weight'] = data['Item_Weight'].fillna(data['Item_Identifier'].map(unique_items_weight))


In [9]:
## Get outlet sizes
data['Outlet_Size'].value_counts(dropna=False)

Outlet_Size
Medium    2793
NaN       2410
Small     2388
High       932
Name: count, dtype: int64

In [10]:
## Consider all the grocery stores as small sizes 
print("Before altering",data[data['Outlet_Size'].isna() & (data['Outlet_Type'] == 'Grocery Store')].shape)
data.loc[ data['Outlet_Size'].isna()  & ( data['Outlet_Type'] == 'Grocery Store' ),'Outlet_Size'] = 'Small'
print("After altering",data[data['Outlet_Size'].isna() & (data['Outlet_Type'] == 'Grocery Store')].shape)

Before altering (555, 12)
After altering (0, 12)


In [11]:
data.isnull().sum()

Item_Identifier                 0
Item_Weight                   354
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1855
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [12]:
data.loc[( data['Outlet_Type'] == 'Supermarket Type1' ) & ( data['Outlet_Location_Type'] == 'Tier 2' ), 'Outlet_Size'] = 'Small'

In [13]:
data.isnull().sum()

Item_Identifier                0
Item_Weight                  354
Item_Fat_Content               0
Item_Visibility                0
Item_Type                      0
Item_MRP                       0
Outlet_Identifier              0
Outlet_Establishment_Year      0
Outlet_Size                    0
Outlet_Location_Type           0
Outlet_Type                    0
Item_Outlet_Sales              0
dtype: int64

In [14]:
cleaned_data = data[~data['Item_Weight'].isna() ]
cleaned_data.to_csv('cleaned_data.csv', index=False)
